# Table of contents

1. [Introduction](#1)
2. [Starting Point](#2) <br>
    2.1 [Constants](#2.1) <br>
    2.2 [Fundamental Ship Parameters](#2.2) <br>
    2.3 [Derived Ship Parameters](#2.3) <br>
3. [Optimization](#3) <br>
    3.1 [Initial Guess | x0](#3.1) <br>
    3.2 [Constraints | constraints](#3.2) <br>
    3.3 [Boundaries | bounds](#3.3) <br>
    3.4 [Objective Function | func](#3.4) <br>
    3.5 [Results](#3.5) <br>

## 1. Introduction <a name="1"></a>

The purpose of this exercise is to optimize the main dimensions and speed of service of a bulk carrier, in order to minimize the Annual Unit Load Shipping cost ($/ton). To solve the exercise, the computational model used - a set of empirical and semi-empirical formulas - is the one proposed by Xuebin (2009), as presented in the image below:

<br />
<img src="./images/computational_model_xuebin.png" width="500" /> 
<br />

With the following given constraints:

<br />
<img src="./images/constrains.png" width="200" /> 
<br />


To apply the optimization, we set as our goal to minimize the transportation cost, which is the ratio of annual costs to annual cargo (i.e, transportation cost = annual costs/annual cargo). To solve this problem with programming, we use the Python Scipy package, which provides fundamental algorithms for scientific computing in Python. Specifically, we use the minimize function provided by this package, whose documentation can be found at https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html

If any error indicating the need to install the Scipy packages occurs when running this notebook, uncomment the final part of the cell below and run it to install it.

In [1]:
# The sys module provides various functions and variables that are used to
# manipulate different parts of the Python runtime environment.
# This provides a safer way to install packages.

# import sys
# !{sys.executable} -m pip install scipy

## 2. Starting Point <a name="2"></a>

Before actually using the minimize function to apply the optimization, we define the set of relevant constants and parameters with their respective initial values:

### 2.1 Constants <a name="2.1"></a>

In [24]:
g = 9.801 # Gravitational acceleration at the earth's surface

### 2.2 Fundamental Ship Parameters <a name="2.2"></a>

In [26]:
L = 55 # Length
B = 5 # Breadth
D = 10 # Depth
T = 5 # Draught
Cb = 0.7 # Block Coefficient
Vk = 14 # TODO - what is Vk?
DWT = 100000 # Deadweight

### 2.3 Derived Ship Parameters  <a name="2.3"></a>

It is important to note that all other parameters are derived from the constants and ship parameters defined above. So, ultimately seeking to calculate the transportation cost, many intermediate parameters are calculated. Nevertheless, some of these parameters are relevant to the application of constraints and are therefore must be defined globally here:

In [27]:
KB = 0.53*T # Vertical Centre of Buoyancy
BMt = (0.085*Cb - 0.002)*(B ** 2)/(T+Cb) # Metacentric Radius
KG = 1 + 0.52*D # Vertical Centre of Gravity
GMt = KB + BMt - KG # Metacentric Height

V = 0.5144*Vk # TODO: is this the service speed?
froude_number = V / ((g*L) ** 0.5)

## 3. Optimization

Now we proceed to apply the minimize function. A more detailed guide to its use can be found at https://realpython.com/python-scipy-cluster-optimize/. But, for the application in question, we can summarize such a function as a function that takes five (05) arguments:

1. **x0**: a list/vector defining the set of parameters (which will be changed in the optimization process) and their respective initial values (also know as the "initial guess" vector);
2. **constraints**: a list of dictionaries defining constraints for possible parameter values obtained in the optimization;
3. **bounds**: a list of limits (maximum and minimum values) for possible parameter values.
4. **fun**: a function that returns the value to be minimized;
5. **method**: the optimization method/algorithm used. In this case, we use the Sequential Least Squares Programming (SLSQP) algorithm. See more here: https://docs.scipy.org/doc/scipy/reference/optimize.minimize-slsqp.html#optimize-minimize-slsqp;

In mathematical terms, the minimize function is solving the following problem:

<br />
<img src="./images/minimize_function.png" width="500" /> 
<br />

Where $F(x)$ corresponds to the **fun** argument; $X$ corresponds to the param vector, whose order of elements follows from the definition of the initial guess, **x0**; $C_j(X) = 0$ is a given equality constraint and $C_j(X) >= 0$ an inequality constraint, which are defined in the **constraints** argument; and $XL <= x <= XU$ is the set of boundaries (L being the lower bound and U the upper one), which are defined in the **bounds** argument.

From the order of arguments above, we build each of them step-by-step for the analyzed case, starting with **x0**.

### 3.3 Initial Guess | x0  <a name="3.3"></a>

The order in which we place each parameter is arbitrary for program implementation. For organizational purposes, however, we put fundamental ship parameters at the beginning and then derived parameters at the end, so the initial guess vector is:

In [13]:
x0 = [L, B, D, T, Cb, Vk, DWT, GMt, froude_number]

It is important to note that, as we will see below, the order of parameters in the initial guess vector **x0** will be relevant in the understanding of other parameters.

### 3.2 Constraints | constraints  <a name="3.2"></a>

We define a sequence of python dictionaries, each representing a constraint of type $C_j(X) >= 0$ (in this case, we only deal with inequality constraints) and with the keys "type" (to define if it is an equality or inequality constraint) and "func" (to define the constraint itself).

In [12]:
def build_constraints():
    '''A function that builds constraints relating two or more parameters
    defined for the optimization problem.


    Returns
    -------
    out : sequence, dict
        Sequence of dictionaries, each representing a specif constraint
        and containing the keys "type" and "fun".

        "type" : {"eq", "ineq"}
            Determines if it is an equality or inequality constraint 

        "fun" : callable
            The function defining the constraint. It receives a vector x with
            all parameters (in the order as defined by the initial guess, x0).
            Equality constraint means that the constraint function result is to
            be zero (fun(x) == 0) whereas inequality means that it is to be 
            non-negative (f(x) >= 0).

    See Also
    --------
    https://stackoverflow.com/questions/42303470/scipy-optimize-inequality-constraint-which-side-of-the-inequality-is-considere/42304099

    Notes
    -----
    Constraints typically don't need a builder function like this and are 
    more succinctly defined with "func" being a lambda function. The decision to
    create the build_constraints function was to try to make it easier to understand
    how constraints are defined in Scipy and to clarify where each constraint of the
    problem was defined.
    '''

    def L_and_B(x):
        '''L/B >= 6'''
        L, B = x[0], x[1]
        return L/B - 6

    def L_and_D(x):
        '''L/D  <= 15'''
        L, D = x[0], x[2]
        return 15 - L/D

    def L_and_T(x):
        '''L/T <= 19'''
        L, T = x[0], x[3]
        return 19 - L/T

    def T_and_DWT(x):
        '''T <= 0.45DWT^0,31'''
        T, DWT = x[3], x[6]
        return 0.45*DWT*(DWT**0.32) - T

    def T_and_D(x):
        '''T <= 0.7D + 0.7'''
        D, T = x[2], x[3]
        return 0.7*D + 0.7 - T

    def stability_condition(x):
        '''GMt >= 0.07B'''
        B, GMt = x[1], x[7]
        return GMt - 0.07*B

    return (
        {'type': 'ineq', 'fun': L_and_B},
        {'type': 'ineq', 'fun': L_and_D},
        {'type': 'ineq', 'fun': L_and_T},
        {'type': 'ineq', 'fun': T_and_DWT},
        {'type': 'ineq', 'fun': T_and_D},
        {'type': 'ineq', 'fun': stability_condition},
    )

### 3.3 Boundaries | bounds  <a name="3.3"></a>

We define a sequence of python tuples, each representing a boundary of type $L <= x <= U$. In this case, the order of the boundaries given in the sequence must match the order of the parameters given in the initial guess vector **x0**.

In [19]:
def build_bounds():
    '''A function that defines bounds for each parameter of the optimization
   problem.


    Returns
    -------
    out : sequence, tup
        Sequence of tuples, each representing the minimum and maximum bounds
        for a specific parameter.

    Notes
    -----
    The order of the bounds of the returned sequence must match the order of
    the initial guess vector (x0) defined. The value "None" represents the
    absence of a minimum or maximum bound.
    '''

    L_bound = (0, 274.32)
    B_bound = (0, None)
    D_bound = (0, None)
    T_bound = (0, None)
    Cb_bound = (0.63, 0.75)
    Vk_bound = (14, 18)
    DWT_bound = (25000, 500000)
    GMt_bound = (None, None)
    froude_number_bound = (0, 0.32)
    return (L_bound, B_bound, D_bound,
            T_bound, Cb_bound, Vk_bound,
            DWT_bound, GMt_bound, froude_number_bound)

### 3.4 Objective Function | func  <a name="3.4"></a>

A function that, from a set of parameters that can be changed (respecting constraints and limits), returns the calculated value of the transport cost, which we seek to minimize. 

In [20]:
def transportation_cost_function(x):
    '''A function that calculates the transportation cost from a set
    of parameters.

    Parameters
    -------
    x : list
        A list of parameters whose order matches what was defined in 
        the initial guess vector (x0)

    Returns
    -------
    out : float
        The value of the transportation cost calculated from given parameters
    '''
    L, B, D, T, Cb, Vk, DWT, GMt, froude_number = x

    # Power
    V = 0.5144*Vk
    displacement = 1.025*L*B*T*Cb
    a = 4977.06*(Cb ** 2) - 8105.61*Cb + 4456.51
    b = -10847.2*(Cb ** 2) + 12817*Cb - 6960.32
    power = (displacement ** (2/3))*(V ** 3) / (a + b*froude_number)

    # Weights
    steel_weight = 0.034*(L**1.7)*(B**0.7)*(D**0.4)*(Cb**0.5)
    outfit_weight = (L ** 0.8)*(B ** 0.6)*(D ** 0.3)*(Cb ** 0.1)

    # We commented the lines containing the variables below because, although
    # they were mentioned in problem assignment, they are not used in our program.
    # machinery_weight = 0.17*(power ** 0.9)
    # ligthship_weight = steel_weight + outfit_weight + machinery_weight

    # General (TODO: give a better name for this category of calculations)
    round_trip_miles = 5000
    sea_days = round_trip_miles/(24*Vk)
    daily_consumption = 0.19*power*24/1000 + 0.2
    fuel_carried = daily_consumption*(sea_days+5)
    miscellaneous_DWT = 2*(DWT ** 0.5)
    cargo_DWT = DWT-fuel_carried-miscellaneous_DWT
    handling_rate = 8000
    port_days = 2*((cargo_DWT/handling_rate)+0.5)
    round_trips_per_year = 350/(sea_days+port_days)

    # Costs
    port_cost = 6.3*(DWT ** 0.8)
    fuel_price = 100
    fuel_cost = 1.05*daily_consumption*sea_days*fuel_price
    voyage_costs = (fuel_cost+port_cost)*round_trips_per_year
    running_cost = 40000*(DWT ** 0.3)
    ship_cost = 1.3*(2000*steel_weight + 3500 *
                     outfit_weight + 2400*(power ** 0.8))
    capital_costs = 0.2*ship_cost

    annual_cost = capital_costs + running_cost + voyage_costs
    annual_cargo = cargo_DWT*round_trips_per_year

    transportation_cost = annual_cost/annual_cargo
    return transportation_cost

### 3.5 Results  <a name="3.5"></a>

Finally, we use the values for each parameter developed as arguments of the minimize function:

In [17]:
from scipy.optimize import minimize

minimize(
    x0=x0, # Initial guess vector
    fun=transportation_cost_function, # Function to be optimized
    method='SLSQP', # We are using the Sequential Least Squares Programming algorithm
    constraints=build_constraints(), # Sequence of dictionaries defining the constraints
    bounds=build_bounds() # Sequence of tuples defining lower and upper bounds for each parameter
)

     fun: 1.997323626655561
     jac: array([ 0.00000000e+00,  9.83130640e+01,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -2.35243738e-02, -1.80304050e-06,  0.00000000e+00,
        0.00000000e+00])
 message: 'Optimization terminated successfully'
    nfev: 90
     nit: 10
    njev: 9
  status: 0
 success: True
       x: array([5.43743348e+01, 5.27355937e-14, 9.03830163e+00, 4.83417188e+00,
       6.30000000e-01, 1.80000000e+01, 1.00000000e+05, 2.31030732e+00,
       0.00000000e+00])

The result (that is, the minimum transportation cost value obtained, is the "fun" value displayed when executing the above ballot.